In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
spark = SparkSession.builder.appName('Amenities Processing').getOrCreate()
    

25/01/24 18:05:25 WARN Utils: Your hostname, w3e103 resolves to a loopback address: 127.0.1.1; using 192.168.0.243 instead (on interface enp2s0)
25/01/24 18:05:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/24 18:05:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/24 18:05:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df=spark.read.json("expedia-lodging-amenities-en_us-1-all.jsonl")


25/01/24 18:05:58 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
df.show(50, truncate= False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
df.printSchema()

root
 |-- cleanlinessAndSafety: struct (nullable = true)
 |    |-- CLEANLINESS: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- DISCLAIMER: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- SAFETY: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- SOCIAL_DISTANCING: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- country: string (nullable = true)
 |-- popularAmenities: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- propertyAmenities: struct (nullable = true)
 |    |-- ACCESSIBILITY: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ACTIVITIES_NEARBY: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- BATHROOMS: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- BEACH: array (nullable = true)
 |    |    |-

In [6]:
df.columns

['cleanlinessAndSafety',
 'country',
 'popularAmenities',
 'propertyAmenities',
 'propertyId',
 'propertyType',
 'roomAmenities']

In [15]:
# from pyspark.sql import functions as F

# # Extracting all arrays from the `propertyAmenities` struct and concatenating them
# df_transformed = df.withColumn(
#     "propertyAmenities",
#     F.flatten(F.array(*[F.col(f"propertyAmenities.{col}") for col in df.select("propertyAmenities.*").columns]))
# )

# df_transformed.show(truncate=False)


In [7]:
df.show(5, truncate= False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
df.printSchema()

root
 |-- cleanlinessAndSafety: struct (nullable = true)
 |    |-- CLEANLINESS: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- DISCLAIMER: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- SAFETY: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- SOCIAL_DISTANCING: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- country: string (nullable = true)
 |-- popularAmenities: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- propertyAmenities: struct (nullable = true)
 |    |-- ACCESSIBILITY: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ACTIVITIES_NEARBY: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- BATHROOMS: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- BEACH: array (nullable = true)
 |    |    |-

In [9]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, StringType

def combine_all_amenities(df):
    """
    Combines and flattens both propertyAmenities and roomAmenities structs into a single array,
    removing any null elements.
    
    Args:
        df: DataFrame containing both propertyAmenities and roomAmenities struct columns
        
    Returns:
        DataFrame with a new column 'all_amenities' containing all non-null amenities in a single array
    """
    # Define arrays for both structs
    room_arrays = [
        'BATHROOM', 'BEDROOM', 'ENTERTAINMENT', 
        'FOOD_AND_DRINK', 'MORE', 'OUTDOOR_SPACE'
    ]
    
    property_arrays = [
        'ACCESSIBILITY', 'ACTIVITIES_NEARBY', 'BATHROOMS', 'BEACH', 'BEDROOM',
        'BUSINESS_SERVICES', 'CLIMATE_CONTROL', 'CONVENIENCES', 'DINING',
        'ENTERTAINMENT', 'FAMILY_FRIENDLY', 'FOOD_AND_DRINK', 'GENERAL',
        'GUEST_SERVICES', 'INTERNET', 'KITCHEN', 'LANGS_SPOKEN', 'LAUNDRY',
        'LIVING_SPACES', 'LOCATION_HIGHLIGHTS', 'MORE', 'OUTDOOR', 'OUTDOORS',
        'PARKING', 'PETS', 'POOL/SPA', 'SAFETY', 'SERVICES_AND_CONVENIENCES',
        'SKI', 'SPA', 'SUITABILITY/ACCESSIBILITY', 'THINGS_TO_DO', 'WORKSPACES'
    ]
    
    # Start with the first array from roomAmenities
    result = F.filter(
        F.coalesce(F.col(f"roomAmenities.{room_arrays[0]}"), F.array()),
        lambda x: x.isNotNull()
    )
    
    # Add remaining room amenities
    for col in room_arrays[1:]:
        filtered_array = F.filter(
            F.coalesce(F.col(f"roomAmenities.{col}"), F.array()),
            lambda x: x.isNotNull()
        )
        result = F.array_union(result, filtered_array)
    
    # Add property amenities
    for col in property_arrays:
        filtered_array = F.filter(
            F.coalesce(F.col(f"propertyAmenities.{col}"), F.array()),
            lambda x: x.isNotNull()
        )
        result = F.array_union(result, filtered_array)

    
    
    # Return DataFrame with new column containing all unique non-null amenities
    return df.withColumn(
        "amenities",
        F.array_distinct(result)
    )

In [10]:
df = combine_all_amenities(df)

In [11]:
df.show(10, truncate= False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [12]:
df.printSchema()

root
 |-- cleanlinessAndSafety: struct (nullable = true)
 |    |-- CLEANLINESS: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- DISCLAIMER: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- SAFETY: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- SOCIAL_DISTANCING: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- country: string (nullable = true)
 |-- popularAmenities: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- propertyAmenities: struct (nullable = true)
 |    |-- ACCESSIBILITY: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ACTIVITIES_NEARBY: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- BATHROOMS: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- BEACH: array (nullable = true)
 |    |    |-

In [13]:
df = df.withColumn(
    "amenities_snake_case",
    F.expr(
        """
        transform(
            amenities,
            x -> lower(
                regexp_replace(
                    regexp_replace(
                        regexp_replace(
                            regexp_replace(
                                regexp_replace(
                                    regexp_replace(
                                        trim(x), 
                                        r"\s*\([^)]*\)", 
                                        ""
                                    ),
                                    " - ", 
                                    "-"
                                ),
                                " ", "_"  
                            ),
                            "-", "_"
                        ),
                        "/", "_"
                    ),
                    '_+', '_'
                )
            )
        )
        """
    )
)


df.show(5, truncate= False)

<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_15259/2612021935.py:4: SyntaxWarning: invalid escape sequence '\s'
  """


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [15]:
import json

# Load the amenity_category.json file into a dictionary
with open("amenity_category.json", "r") as f:
    amenity_category_dict = json.load(f)

flat_amenity_category_dict = {}
for amenity, catagories in amenity_category_dict.items():
    if isinstance(catagories, list):
        for catagory in catagories:
            flat_amenity_category_dict[amenity] = catagory
    else:
        flat_amenity_category_dict[amenity] = catagories



In [16]:
from pyspark.sql import functions as F

# Convert the flat_amenity_category_dict into a Spark map
mapping_expr = F.create_map(
    *[F.lit(key_val) for key_val in sum(flat_amenity_category_dict.items(), ())]
)

# Map each amenity to its category and ensure unique categories
df = df.withColumn(
    "categories",
    F.array_distinct(  # Ensure only unique categories
        F.transform(
            F.col("amenities_snake_case"),
            lambda amenity: F.coalesce(F.element_at(mapping_expr, amenity), F.lit("Other"))
        )
    )
)
df.show(5,truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------